In [1]:
!nvidia-smi

Thu Feb  1 03:15:57 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   37C    P0    35W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install accelerate bitsandbytes langchain

In [3]:
%cd /content
!apt-get update -y 
!apt-get -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00001-of-00002.safetensors -d /content/models/Llama-2-7b-chat-hf -o model-00001-of-00002.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00002-of-00002.safetensors -d /content/models/Llama-2-7b-chat-hf -o model-00002-of-00002.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/model.safetensors.index.json -d /content/models/Llama-2-7b-chat-hf -o model.safetensors.index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/special_tokens_map.json -d /content/models/Llama-2-7b-chat-hf -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/tokenizer.model -d /content/models/Llama-2-7b-chat-hf -o tokenizer.model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/tokenizer_config.json -d /content/models/Llama-2-7b-chat-hf -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/config.json -d /content/models/Llama-2-7b-chat-hf -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/generation_config.json -d /content/models/Llama-2-7b-chat-hf -o generation_config.json


/content
Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://deb.nodesource.com/node_16.x focal InRelease                     
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease             
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Reading package lists... Done

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
5fefb8|OK  |       0B/s|/content/models/Llama-2-7b-chat-hf/model-00001-of-00002.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
3c7b18|OK  |       0B/s|/content/mode

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch

from transformers import BitsAndBytesConfig
from torch import bfloat16

# Our 4-bit configuration to load the LLM with less GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

model_id = "/content/models/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, revision='main', use_fast=True, trust_remote_code=True)
model_config = AutoConfig.from_pretrained(
    model_id
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [12]:

from transformers import TextIteratorStreamer
from threading import Thread

def chat(messages, model, tokenizer, **kwargs):
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    input_ids = input_ids.to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=60.0, skip_prompt=True, skip_special_tokens=True)
    
    gen_kwargs = {"input_ids": input_ids, "max_length": 3512, "eos_token_id": tokenizer.eos_token_id, "streamer": streamer, 'do_sample': True, 'temperature': None, 'top_k': 10}
    thread = Thread(target=model.generate, kwargs=gen_kwargs)
    thread.start()
    return streamer

def display_chat(prompt, system=None, ret=False, **kwargs):
    messages = [{'role': 'user', 'content': prompt}]
    if system is not None:
        messages = [{'role': 'system', 'content': system}] + messages

    message = ""
    for word in chat(messages, model, tokenizer):
        print(word, end='')
        message += word
    
    if ret:
        return message


In [6]:
PROMPT_TEMPLATE = """"You are an math expert which solve questions best as you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
OBSERVATION: the result of the action
... (this Thought/Action/Action Input/OBSERVATION can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

Question: {query}"""
display_chat(PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"))


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 Sure, I'm happy to help! Here's how I would approach this problem:

Thought: Hmm, let's see... when we divide 2 by 2, we get 1. So, if we multiply 8 by 2, we get 16. Now, let's divide 16 by 2, which gives us 8.

Action: Let's calculate the final answer.
Action Input: 8 + 2 / 2 * 2 = ?

OBSERVATION: Hmm, that gives us 16.

Thought: Great, we have our final answer! The final answer is 16.
Final Answer: 16

In [7]:
iterations = 3
results = []
for _ in range(iterations):
    ret = display_chat(PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"), ret=True)
    results.append(ret)
results

 Sure, I'm happy to help you with that problem! Here's how I would approach it:

Thought: Okay, let's break this problem down step by step. We have 8 + 2, which is equal to 10. Now, we have 2 / 2, which is equal to 1. So, the total is 10 + 1, which is equal to 11.

Action: I will calculate 11 multiplied by 2.
Action Input: 11 multiplied by 2 is equal to 22.

OBSERVATION: The result of multiplying 11 by 2 is 22.

Thought: Great, now we have a total of 22. Let's check our answer by adding 8 and 2.

Action: I will calculate 8 + 2.
Action Input: 8 + 2 is equal to 10.

OBSERVATION: The result of adding 8 and 2 is 10.

Thought: Perfect! Now we have confirmed that our answer is correct. The final answer to the original input question is 10.

Final Answer: 10. Sure! I'm happy to help you with your math problem. Here's how I would approach this question:

Thought: Let's break down the problem step by step. We have 8 + 2 = 10, and then we have 2 * 2 = 4. So far, we have 10 + 4 = 14.

Action: I w

[" Sure, I'm happy to help you with that problem! Here's how I would approach it:\n\nThought: Okay, let's break this problem down step by step. We have 8 + 2, which is equal to 10. Now, we have 2 / 2, which is equal to 1. So, the total is 10 + 1, which is equal to 11.\n\nAction: I will calculate 11 multiplied by 2.\nAction Input: 11 multiplied by 2 is equal to 22.\n\nOBSERVATION: The result of multiplying 11 by 2 is 22.\n\nThought: Great, now we have a total of 22. Let's check our answer by adding 8 and 2.\n\nAction: I will calculate 8 + 2.\nAction Input: 8 + 2 is equal to 10.\n\nOBSERVATION: The result of adding 8 and 2 is 10.\n\nThought: Perfect! Now we have confirmed that our answer is correct. The final answer to the original input question is 10.\n\nFinal Answer: 10.",
 " Sure! I'm happy to help you with your math problem. Here's how I would approach this question:\n\nThought: Let's break down the problem step by step. We have 8 + 2 = 10, and then we have 2 * 2 = 4. So far, we hav

In [13]:
SELF_REFLECT_PROMPT = """
### User input ###
{initial_prompt}:
### Result ###
{results}

Reflect about the result regarding the user input and give the final answer
"""
display_chat(
    SELF_REFLECT_PROMPT.format(
        initial_prompt=PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"),
        results="\n".join(results)
    )
)


 Sure, I'm happy to help you with that problem! The user input is "how much is 8 + 2 / 2 * 2?".

To solve the problem, I will follow the steps below:

Thought: Okay, let's break this problem down step by step. We have 8 + 2, which is equal to 10. Now, we have 2 / 2, which is equal to 1. So, the total is 10 + 1, which is equal to 11.

Action: I will calculate 11 multiplied by 2.
Action Input: 11 multiplied by 2 is equal to 22.

OBSERVATION: The result of multiplying 11 by 2 is 22.

Thought: Great, now we have a total of 22. Let's check our answer by adding 8 and 2.

Action: I will calculate 8 + 2.
Action Input: 8 + 2 is equal to 10.

OBSERVATION: The result of adding 8 and 2 is 10.

Thought: Perfect! Now we have confirmed that our answer is correct. The final answer to the original input question is 10.

Final Answer: 10.

Therefore, the final answer to the question "how much is 8 + 2 / 2 * 2?" is 10.

In [15]:
print(SELF_REFLECT_PROMPT.format(
        initial_prompt=PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"),
        results="\n".join(results)
))


### User input ###
"You are an math expert which solve questions best as you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
OBSERVATION: the result of the action
... (this Thought/Action/Action Input/OBSERVATION can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

Question: how much is 8 + 2 / 2 * 2?:
### Result ###
 Sure, I'm happy to help you with that problem! Here's how I would approach it:

Thought: Okay, let's break this problem down step by step. We have 8 + 2, which is equal to 10. Now, we have 2 / 2, which is equal to 1. So, the total is 10 + 1, which is equal to 11.

Action: I will calculate 11 multiplied by 2.
Action Input: 11 multiplied by 2 is equal to 22.

OBSERVATION: The result of multiplying 11 by 2 is 22.

Thought: Great, now we have a total